In [2]:
import cv2
import numpy as np
import psutil
import os
from tqdm.contrib.concurrent import thread_map
from tqdm.auto import tqdm
from tkinter import Tk, filedialog
from multiprocessing import cpu_count
from multiprocessing.dummy import Pool as ThreadPool

p = psutil.Process(os.getpid())
p.cpu_affinity([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])


# 選取資料檔案
root = Tk()
root.attributes('-topmost', True)  # 將主視窗設置為最上層
root.withdraw()

data_file_paths = filedialog.askopenfilenames(parent = root ,title='選擇資料檔案')

filename = data_file_paths


#output
width, height = 480, 480
fps = 30
savefilename = os.path.basename(filename[0])
# 輸出資料夾設定
output_dir = filedialog.askdirectory(title='選擇輸出資料夾')
ROI_num = 4
start_time = [12, 546, 966, 1516] #sec
duration = 30 #min
ispreview = False


# 創建VideoCapture對象
cap = cv2.VideoCapture(filename[0])
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 檢查文件是否成功打開
if not cap.isOpened():
    print('無法打開文件')

# 建立 VideoWriter 物件
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 創建窗口並設置鼠標回調函數
cv2.namedWindow('frame')
points = []
Ms = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        if len(points) == ROI_num*4:
            cv2.setMouseCallback('frame', lambda *args: None)

cv2.setMouseCallback('frame', mouse_callback)

previewed = False

# 開始讀取視頻幀
while True:
    ret, frame = cap.read()
    
    # 當讀取完所有幀，退出循環
    if not ret:
        break
        
    resized_frame = cv2.resize(frame, (1600, 900))
        
    # 在每次點擊時繪製圓圈
    for point in points:
        cv2.circle(resized_frame, point, 5, (0, 0, 255), -1)
        
    # 等待用戶選擇四個點
    if len(points) < ROI_num*4:
        cv2.imshow('frame', resized_frame)
        cv2.waitKey(1)
        continue
        
    # 計算變換矩陣
    dst_points = np.float32([[0, 0], [height, 0], [height, width], [0, width]])
    
    for i in range(ROI_num):
        src_points = np.float32((points[0+i*4],points[1+i*4],points[2+i*4],points[3+i*4]))
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        Ms.append(M)
    break
    

def process_roi(i):
    try:
        outputfull = os.path.join(output_dir, savefilename + f'_ROI_{i}.mp4')
        out = cv2.VideoWriter(outputfull, fourcc, fps, (width, height))
        cap = cv2.VideoCapture(filename[0])
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_time[i]*fps)
        now_frames = 0
        end_frame = (60*duration)*fps
        
        for _ in tqdm(range(end_frame), desc=f"Processing ROI {i}"):
            ret, frame = cap.read()
            
            if not ret or now_frames >= end_frame:
                break
                
            resized_frame = cv2.resize(frame, (1600, 900))
            warped = cv2.warpPerspective(resized_frame, Ms[i], (height, width))
            resized_frame = cv2.resize(warped, (480, 480))
            
            out.write(resized_frame)
            now_frames += 1

    except Exception as e:
        print(f"Error in thread {i}: {e}")
    finally:
        # 確保資源被釋放
        cap.release()
        out.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    thread_map(process_roi, range(ROI_num), max_workers=cpu_count())
    cv2.destroyAllWindows()

  0%|          | 0/4 [00:00<?, ?it/s]

Processing ROI 0:   0%|          | 0/54000 [00:00<?, ?it/s]

Processing ROI 3:   0%|          | 0/54000 [00:00<?, ?it/s]

Processing ROI 1:   0%|          | 0/54000 [00:00<?, ?it/s]

Processing ROI 2:   0%|          | 0/54000 [00:00<?, ?it/s]